<a href="https://colab.research.google.com/github/EnesGokceDS/Score_Prediction/blob/main/Best_Egg_bert_multi_class_classification_with_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **In this notebook, I approached the BERT-pretaining task with a sligtly different way.**
Main differences are:
- Tensorflow
- Categorical Crossentropy in the loss function

In [ ]:
%%capture
pip install transformers

In [ ]:
!git clone https://github.com/MarletteFunding/marlette-ds-challenge2

Cloning into 'marlette-ds-challenge2'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 7 (delta 0), reused 3 (delta 0), pack-reused 4
Unpacking objects: 100% (7/7), done.


In [ ]:
import pandas as pd
df_train = pd.read_csv('marlette-ds-challenge2/NLP_task_train.csv.zip',header=0,index_col=0,compression='infer')
df_validate = pd.read_csv('marlette-ds-challenge2/NLP_task_validate.csv.zip',header=0,index_col=0,compression='infer')

In [ ]:
df_train["SENTENCE"] = df_train["SENTENCE"].astype(str)

In [ ]:
import numpy as np

In [ ]:
seq_len = 512

In [ ]:
num_samples = len(df_train)

In [ ]:
Xids = np.zeros((num_samples, seq_len))

In [ ]:
Xmask = np.zeros((num_samples, seq_len))

In [ ]:
Xids.shape

(56644, 512)

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
for i, phrase in enumerate(df_train["SENTENCE"]):
  tokens = tokenizer.encode_plus(phrase, max_length=seq_len, truncation=True,
                                 padding="max_length", add_special_tokens=True, 
                                 return_tensors="tf")
  Xids[i, :] = tokens["input_ids"]
  Xmask[i, :] = tokens["attention_mask"]

In [ ]:
Xids

array([[  101.,  1109.,  2006., ...,     0.,     0.,     0.],
       [  101.,  1135.,  1108., ...,     0.,     0.,     0.],
       [  101.,  1555.,   102., ...,     0.,     0.,     0.],
       ...,
       [  101.,  3982.,  1304., ...,     0.,     0.,     0.],
       [  101.,  1693., 17067., ...,     0.,     0.,     0.],
       [  101.,   146.,  8856., ...,     0.,     0.,     0.]])

In [ ]:
Xmask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [ ]:
df_train["SCORE"] = df_train["SCORE"].astype(int)

In [ ]:
arr = df_train["SCORE"].values

In [ ]:
arr

array([10, 10, 10, ..., 10,  8, 10])

In [ ]:
labels = np.zeros((num_samples, arr.max()+1))

In [ ]:
labels.shape

(56644, 11)

In [ ]:
labels[np.arange(num_samples), arr] = 1

In [ ]:
labels

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [ ]:
dataset.take(1)

<TakeDataset shapes: ((512,), (512,), (11,)), types: (tf.float64, tf.float64, tf.float64)>

In [ ]:
labels[0,:].shape

(11,)

In [ ]:
def map_func(input_ids, masks, labels):
  return {"input_ids": input_ids, "attention_mask": masks}, labels

In [ ]:
dataset = dataset.map(map_func)

In [ ]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (512,), attention_mask: (512,)}, (11,)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [ ]:
batch_size = 16

In [ ]:
df_validate = df_validate.dropna(subset=["SCORE"])

In [ ]:
df_train.SCORE.value_counts()

10    43843
9      6214
8      3632
7      1297
5       516
6       434
0       227
4       192
3       127
1        83
2        79
Name: SCORE, dtype: int64

In [ ]:
df_validate.SCORE.value_counts()

10.0    10866
9.0      1563
8.0       905
7.0       332
5.0       111
6.0        97
4.0        46
0.0        35
3.0        26
2.0        20
1.0        17
Name: SCORE, dtype: int64

In [ ]:
df_validate["SCORE"] = df_validate["SCORE"].astype(int)
df_validate["SENTENCE"] = df_validate["SENTENCE"].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
num_samples_validate = len(df_validate)

In [ ]:
Xids_validate = np.zeros((num_samples_validate, seq_len))
Xmask_validate = np.zeros((num_samples_validate, seq_len))

In [ ]:
arr_validate = df_validate["SCORE"].values
labels_validate = np.zeros((num_samples_validate, arr_validate.max()+1))
labels_validate[np.arange(num_samples_validate), arr_validate] = 1

In [ ]:
for i, validate in enumerate(df_validate["SENTENCE"]):
  tokens_validate = tokenizer.encode_plus(validate, max_length=seq_len, truncation=True,
                                 padding="max_length", add_special_tokens=True, 
                                 return_tensors="tf")
  Xids_validate[i, :] = tokens["input_ids"]
  Xmask_validate[i, :] = tokens["attention_mask"]

In [ ]:
labels_validate

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
dataset_validate = tf.data.Dataset.from_tensor_slices((Xids_validate, Xmask_validate, labels_validate))

In [ ]:
dataset_validate = dataset_validate.map(map_func)

In [ ]:
dataset = dataset.shuffle(5000).batch(batch_size, drop_remainder=True)
dataset_validate = dataset_validate.shuffle(2000).batch(batch_size, drop_remainder=True)

In [ ]:
size= int((num_samples / batch_size))

In [ ]:
size_validate = int((num_samples_validate / batch_size))

In [ ]:
train_ds = dataset.take(20)  # size * split normally but i will keep it 20 for now

In [ ]:
# train_ds = train_ds.sample(200)

In [ ]:
val_ds = dataset_validate.take(5)

In [ ]:
# val_ds = val_ds.sample(20)

In [ ]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained("bert-base-cased")

bert.summary()

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [ ]:
input_ids = tf.keras.layers.Input(shape=(seq_len,), name= "input_ids" , dtype="int32")
mask = tf.keras.layers.Input(shape=(seq_len,), name= "attention_mask" , dtype="int32")

embeddings = bert.bert(input_ids, attention_mask=mask)[1]

x = tf.keras.layers.Dense(1024, activation="relu")(embeddings)
y = tf.keras.layers.Dense(arr.max()+1, activation="softmax", name="outputs")(x)

In [ ]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy("accuracy")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
history = model.fit(
    train_ds,
    epochs=3,
    validation_data=val_ds
)

Epoch 1/3
20/20 [==============================] - 265s 13s/step - loss: 1.8408 - accuracy: 0.4531 - val_loss: 1.0079 - val_accuracy: 0.8000
Epoch 2/3
20/20 [==============================] - 257s 13s/step - loss: 0.9371 - accuracy: 0.7781 - val_loss: 0.8180 - val_accuracy: 0.8000
Epoch 3/3
20/20 [==============================] - 251s 13s/step - loss: 0.8577 - accuracy: 0.7969 - val_loss: 1.1910 - val_accuracy: 0.6875


In [ ]:
model.save("multi_class_sentiment")

INFO:tensorflow:Assets written to: multi_class_sentiment/assets


INFO:tensorflow:Assets written to: multi_class_sentiment/assets


In [ ]:
df_test = pd.read_csv("/content/sentiment_analysis_reviews_version_1.csv")

In [ ]:
model_test = tf.keras.models.load_model("multi_class_sentiment/")

In [ ]:
model_test.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

def prep_data(text):
  tokens = tokenizer.encode_plus(text, max_length=512,
                                 truncation=True, padding="max_length",
                                 add_special_tokens=True, return_token_type_ids=False,
                                 return_tensors="tf")
  return {"input_ids": tf.cast(tokens["input_ids"], tf.float64),
          "attention_mask": tf.cast(tokens["attention_mask"], tf.float64)
  }

In [ ]:
test = prep_data("this is awful")

In [ ]:
probs = model.predict(test)

In [ ]:
probs[0]

array([0.00864448, 0.0104871 , 0.0035487 , 0.00291142, 0.01227017,
       0.00912691, 0.00634492, 0.01829215, 0.03909151, 0.07232194,
       0.8169607 ], dtype=float32)

# **Interpretation**

Interpretation are similar to pytorch version of the pre-training

* Basically, 81% of the SENTENCES will be labeled as SCORE 10 with this model. 

* In this analysis, 320 SENTENCES were used for easy pre-training.
* Weighted Average score is (68%). Weighted F1 score calculates the F1 score for each class independently but when it adds them together uses a weight that depends on the number of true labels of each class. This is overall performance of the model.

* If we desperately want to obtain a better model performance quickly, we can try Random Forest classifier, Multi-nominal Naive Bayes, and few other multi-label classification algorithms. However, they are all rule-based models. I would never suggest to use them in production. I strongly advice to spend time on building a better transformers-based language model.

# **Ideas for further improvement**

* This prediction model can be trained with stronger/heavier Transformers models such as RoBERTa, T5-XL etc. We can expect performance improvement with different models.

* I set ***batch_size = 16***. This can be done 32 or higher to run the training faster if there is a higher RAM
* Training the model with more fine-tuned parameters have potential give to give better performance.

* I wonder how the prediction performance would be if we use use Zero-Shot Classification approach. It worths giving it a try if we have enough time.

* Of course with a larger training and validation datasets, and the model can be trained better, model performance can be improved.

* As a data scientist, I can question the validity of the data, and try to understand potential biases in the dataset. Is there any threat to validity in the data gathering process? This point is part of a seperate discussion but still can affect the prediction performance.
* All these suggestions needs to be evaluated regarding business needs, upcoming deadlines, stakeholders' priorities. 